# Biofuels vs solar land use

## Introduction

TODO

## Initial setup
### Install and import requirements

To be able to run the code, there are some libraries you may need to install.

In [ ]:
# Whether you run this code in your local computer or on Google Colab, you first need to install the required dependencies:
# * owid-catalog: a library that lets you explore and load datasets from OWID.
# * plotly: a library that lets you create interactive visualizations.
# %pip install --upgrade owid-catalog plotly > /dev/null 2>&1

Import necessary libraries.

In [1]:
import plotly.express as px
from owid.catalog import Table

Set common variables.

In [28]:
# Factor to convert hectares to square meteres.
HA_TO_M2 = 100 * 100
# Factor to convert MWh to TWh.
MWH_TO_TWH = 1e-6

## Cropland used for biofuel production

### UFOP estimate
[The UFOP Report on Global Market Supply 2022/2023](https://www.ufop.de/files/4716/7878/1413/UFOP_Global_Suppy_Report_2022-2023.pdf) (page 41) gives an estimate for 2021.
Adding up the numbers in the chart, we get to a total of:
* 84 million hectares devoted to biofuel production.
Ironically, the title of the chart is "Biofuels take up little space".

The source quoted in the chart is: OECD, USDA, Oil World.
But it also says "©AMI 2022"; so I understand the chart was originally made by AMI based on those sources.
Going to the bibliography, [the AMI data](https://www.ami-informiert.de/ami-shop/shop/detail?ai%5Bd_name%5D=Markt_aktuell_%C3%96lsaaten_und_Bioenergie&ai%5Bd_prodid%5D=110&ai%5Bd_pos%5D=11&ai%5Bcontroller%5D=Catalog&ai%5Baction%5D=detail) seems to not be publicly available.
One of the sources is [the OECD Agricultural Outlook](https://data-explorer.oecd.org/vis?tenant=archive&df[ds]=DisseminateArchiveDMZ&df[id]=DF_HIGH_AGLINK_2017&df[ag]=OECD&dq=..&lom=LASTNPERIODS&lo=5&to[TIME_PERIOD]=false).

There is this page on all [OECD-FAO Agricultural outlook reports](https://www.agri-outlook.org/).
The latest as of now is 2025-2034; [the data](https://data-explorer.oecd.org/vis?fs[0]=Topic%2C1%7CAgriculture%20and%20fisheries%23AGR%23%7CAgricultural%20trade%20and%20markets%23AGR_TRD%23&pg=0&fc=Topic&bp=true&snb=5&si=2&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_AGR%40DF_OUTLOOK_2025_2034&df[ag]=OECD.TAD.ATM&df[vs]=1.1&dq=OECD.A.CPC_0111...&pd=2010%2C2034&to[TIME_PERIOD]=false) (both observations and projections) is publicly available.

It's possible to download the full dataset, but the data about biofuels is not exactly what we need.
They do provide, for each country and year, the tonnes (or litters) of biofuel products from each feedstock item; but they don't provide the tonnes of feedstock required to produce them.
We'd need, for each feedstock item, the conversion factor (from tonnes of biofuel to tonnes of crop).
Then, with that, we could get the corresponding yield for that country-year from FAOSTAT, to convert from tonnes of crop to area.
But where could we get those conversion factors?
As we'll see later, we could get those from different reports from USDA, but they vary at the country level.
Maybe using some average factors could be good enough.

Note that in principle we could also get biofuel production (total, but also biogasoline and biodiesel separately) for each country-year from the Energy Institute Statistical Review of World Energy.
But they don't give data on which feedstock they come from specifically (whereas the OECD data does).

### Cerulogy estimate
An alternative estimate comes from [this report by Cerulogy](https://www.transportenvironment.org/uploads/files/Cerulogy_Diverted-harvest_November_2024.pdf).
Cerulogy estimates 61.3 million hectares of gross biofuel production in 2023.
They then take away a certain percentage of co-products, and estimate an area of:
* 32.0 million hectares of land devoted to net biofuel production.

As they show in their Table 1 (which is based on the Statistical Review), the countries they include cover 95% of the global production (from 9 regions; 5% is assigned to "Rest of the World").
However, 95% of production from those countries (USA, Brazil, EU+UK, Indonesia, China, India, Argentina, Canada, and Thailand) doesn't translate into 95% of land; in fact those countries may import feedstock from other countries.
They do point out this caveat:
"Global trade in biofuel feedstocks (rather than finished fuel), which is not captured by this data, is also sizeable; but it is not trivial to determine from general trade data which feedstocks are used in the biofuels industry versus other industries (cf. Malins & Sandford, 2022)."
Note that second generation biofuels (e.g. from municipal solid waste or bagasse) is not included, but in terms of land use it's probably a small fraction of first generation.

Where do they obtain the feedstock tonnes?
Their values from Table 2 ("Feedstock consumption (Mt) for first-generation biofuels in 2023 for the case study countries") comes from 
"USDA GAIN reports (Danielson, 2023; Das, 2024; Flach et al., 2023; Florence Mojica-Sevilla, 2024; Hayashi, 2024; Joseph, 2024; Prasertsri, 2024; Rahmanulloh, 2023); EIA data publication (U.S. Energy
Information Administration, 2024a); UK RTFO data (UK Department for Transport, 2024c)"
Those sources are listed in the bibliography, but unfortunately, they are individual PDF reports for different countries, e.g. [India](https://www.fas.usda.gov/data/india-biofuels-annual-9), and [Canada](https://apps.fas.usda.gov/newgainapi/api/Report/DownloadReportByFileName?fileName=Biofuels%20Annual_Ottawa_Canada_CA2023-0030.pdf).
These reports seem to also rely on some conversion factors, but they are somewhat different for each country.

# Electricity that could be generated with Solar PV on that land

We can start with the estimates of land use for different energy sources from [OWID's article](https://ourworldindata.org/land-use-per-energy-source) which rely on [this UNECE report](https://unece.org/sites/default/files/2022-04/LCA_3_FINAL%20March%202022.pdf) and [this corrigendum](https://unece.org/sites/default/files/2022-07/Corrigendum%20to%20UNECE%20LCA%20report%20-%20land%20use.pdf).

They do a lifecycle assessment of the amount of land required per year to produce 1 MWh of electricity.
This is estimated over the entire production process (so, including the extraction of materials, refining processes, etc.).

For Solar PV, I'll consider silicon technology only, which is the most commonly used.
The UNECE study considers poly-Si (not mono-Si).
But in terms of land use, I assume they are similar.
Their lifetime land use depends on whether they are installed on-ground, or on roofs.
Given that we want to consider the land currently used for biofuel production, it makes sense to consider only on-ground PV.
Therefore, the minimum, average, and maximum (lifecycle) land used is 12, 19, and 37 (in square meters-annum per MWh).

In [27]:
tb = Table({"Solar PV silicon on-ground": [12, 19, 37]}).transpose().rename(columns={0: "minimum", 1: "average", 2: "maximum"})
# Note that the min and max values comes from Figure 1 of the corrigendum.
# The average comes from the static chart in OWID's article.
# TODO: Where did Hannah get the averages from?


Let's assume for now the Cerulogy estimate of 32 million hectares of land globally used for biofuel production.
If we instead used that land every year for electricity generation from solar PV,
how much electricity could be produced?

In [29]:
biofuels_land = 32e6 * HA_TO_M2
for column in tb.columns:
    value = (biofuels_land * MWH_TO_TWH * 1/tb[column]).item()
    print(f"{column}: {value:.0f} TWh")

minimum: 26667 TWh
average: 16842 TWh
maximum: 8649 TWh


These numbers are surprisingly high.
The maximum generation obtained by replacing biofuels land with solar PV is ~26 PWh, which is very close to the [global electricity production](https://ourworldindata.org/grapher/electricity-prod-source-stacked) in 2024, of about about 30 PWh.

The mid value is 17 PWh, which is also close to [the global electricity from fossil fuels](https://ourworldindata.org/grapher/electricity-fossil-fuels?tab=line&country=~OWID_WRL) in 2024, of about 18 PWh.

Let's try to replicate the numbers quoted in Cerulogy.
According to the Statistical Review, global biofuel consumption was (rounded to 2 significant figures) about 4300 PJ in 2022 (after rounding, the number is identical for biofuel production; also note that Cerulogy quotes 4700, I suppose estimates have changed a bit since then).
How much land would we require to produce that amount of energy with solar PV electricity?

In [30]:
# Factor to convert petajoules to kilowatt-hours.
PJ_TO_KWH = 2.778e+8

The amount of electricity produced by a square meter in a solar farm with 0.08kWp, assuming constant peak performance during a year would be

( 0.08 kWh / 1h ) * ( 365 days / 1 year ) * ( 24 h / 1 day ) = 701 kWh / year / m2

But instead of assuming constant peak performance, they assume a farm productivity of 3.9 kWh/kWp/day, so:

0.08 kWp * ( 3.9 kWh / kWp / day ) * ( 365 days / 1 year ) =  114 kWh / year / m2

In [55]:
# 0.08 * 3.9 * 365

How many million hectares of such a solar farm would be needed to produce 4700 PJ?

( 4700 PJ / year ) * ( 2.778e8 kWh / 1 PJ ) * ( 1 m2 * 1 year / 114 kWh ) * ( 1 ha / 1e4 m2 ) * ( 1 M ha / 1e6 ha ) = 1.1 M ha

In [ ]:
# 4700 * 2.778e8 / 114 / 1e4 / 1e6

So, to produce 4700 PJ with solar farms, only 1.1 M ha are required, which is 3.4% of the 32 M ha needed to produce that energy with biofuels.

In [54]:
# 100 * 1.1 / 32

Conversely, we can also estimate how much electricity could solar farms produce on the 32 M ha used for biofuel production.

32 M ha * ( 1e6 ha / 1 M ha ) * ( 1e4 m2 / 1 ha ) * ( 114 kWh / year / m2 ) * ( 1 TWh / 1e9 kWh ) = 36480 TWh ~ 36 PWh

This is even larger than the maximum obtained with the lifecycle estimates from the OWID article (from UNECE).

It's unclear to me whether the 32 M ha of biofuels are comparable with the lifecycle area estimated for solar, but let's assume that for comparison.
Taking the average value from the OWID article, the amount of land needed to produce 1 MWh with a solar farm (poly-Si) is 19 m2.

( 4700 PJ / year ) * ( 19 m2 * year / MWh ) * ( 2.778e5 MWh / 1 PJ ) * ( 1 ha / 1e4 m2 ) * ( 1 M ha / 1e6 ha ) = 2.4 M ha 

In other words, with solar farms we'd need a 7.5% of the surface required to produce the same amount of energy as biofuels.

So, if we used the full 32 M ha, how much electricity could we produce? (This should be exactly the same number that we calculated before).

32 M ha * ( 1e6 ha / 1 M ha ) * ( 1e4 m2 / 1 ha ) * ( 1 MWh / year / 19 m2 * year) * ( 1 TWh / 1e6 MWh ) = 16842 TWh ~ 17 PWh

In [74]:
# TODO: Consider how these estimates could be refined.

## How much electricity is needed to power all of the world's cars and trucks?

In [70]:
# From the IEA's World Energy Outlook 2025 (free dataset)
# https://www.iea.org/data-and-statistics/data-product/world-energy-outlook-2025-free-dataset
# we can extract the total Road energy in 2024, which is 93.275 EJ.

In [ ]:
# We can safely neglect the fraction of vehicles that is produced with electricity.
# Let's assume an efficiency of around 30% for combustion cars and 80% for electric cars.
# Then, if all vehicles were replaced by electric ones, how much energy would they require?
# 93 EJ * ( 277 TWh / 1 EJ ) * ( 0.30 / 0.80 ) = 9660 TWh ~ 9.7 PWh
93 * 277 * 0.30 / 0.80

9660.374999999998